# 面向对象：封装、继承、多态

## 封装：把数据和操作打包在一起

### 私有化：下划线约定和“改名”机制

封装里最常见的，就是用下划线把“内部用”的东西标记出来，不打算给外部随便用。

In [1]:
class Demo:
    def __init__(self):
        self.public = 1          # 公开属性
        self._inner = 2          # 约定：内部使用
        self.__secret = 3        # 真·私有属性（会被改名）

d = Demo()
print(d.public)          # 1
print(d._inner)          # 2
# print(d.__secret)      # 取消注释会报错：没有这个属性
print(d._Demo__secret)   # 3，绕过私有保护的访问方式

1
2
3


这样写的好处是：清楚标记“这是内部实现细节，别随便碰”，外部调用的人一看就明白哪些东西不该直接用。

### 私有属性：只在类内部使用的数据

私有属性就是前面加了双下划线的实例变量，只打算在类内部读写。

In [2]:
class Person:
    def __init__(self, name):
        self.__name = name

    def get_name(self):
        return self.__name

p = Person("张三")
print(p.get_name())      # 张三
print(p._Person__name)   # 张三，不推荐的访问方式
# print(p.__name)        # 取消注释会报错：没有这个属性

张三
张三


用私有属性的好处是：以后你想改内部实现（比如额外做校验、日志），外部代码不用跟着改，减少“牵一发而动全身”的风险。

### 私有方法：只给类自己调用的小工具函数

私有方法就是前面加了双下划线的实例方法，只打算在类内部被其他方法调用。

In [3]:
class Person:
    # 私有方法
    def __private_method(self):
        print("private method")

    # 对外公开的方法
    def do_something(self):
        self.__private_method()

p = Person()
p.do_something()              # private method
p._Person__private_method()   # private method，不推荐的调用方式
# p.__private_method()        # 取消注释会报错

private method
private method


把内部细节写成私有方法，可以防止别人误调用，只暴露少量“主入口”，代码更稳、更好维护。

### 使用 @property：把方法当属性用

`@property` 能把一个“无参数的方法”伪装成属性，用起来更自然。

In [4]:
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def eat(self):
        print(f"{self.name} is eating...")

p = Person("张三")
p.eat      # 张三 is eating...
# p.eat()  # 取消注释会报错，因为现在它已经不是“函数调用”用法了

张三 is eating...


用 `@property` 的好处是：外部用起来像操作属性一样简单，但内部可以写成一段逻辑，既优雅又能随时加功能。

### 只读属性：外部只能看，不能改

用私有属性存数据，再配合 `@property` 提供只读接口，就做出了“只能读不能写”的属性。

In [5]:
class Person:
    def __init__(self, name):
        self.__name = name      # 私有属性

    @property
    def name(self):
        return self.__name      # 只读

p = Person("张三")
print(p.name)       # 张三
# p.name = "李四"   # 取消注释会报错，无法直接修改

张三


只读属性的好处是：保证一些重要信息（比如身份证号、创建时间）不会被外部胡乱改掉。

### 可读写属性：写入时顺便做校验

在只读属性的基础上，再加一个 `@属性名.setter`，就能控制“被修改”时的行为。

In [6]:
class Person:
    def __init__(self, name):
        self.__name = name

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, name):
        if name == "李四":
            print("不许叫李四")
        else:
            self.__name = name

p = Person("张三")
print(p.name)   # 张三

p.name = "李四" # 不许叫李四
print(p.name)   # 还是 张三

p.name = "王五"
print(p.name)   # 王五

张三
不许叫李四
张三
王五


这样一来，属性赋值这一步就能顺便做校验和拦截，避免出现乱七八糟的数据。

### property 的坑：不要和属性同名

`@property` 修饰的方法名字，如果和你要返回的属性名字一样，会导致无限递归。

In [7]:
class Person:
    @property
    def name(self):
        return self.name   # 一直在调自己

p = Person()
# p.name                # 取消注释会导致递归错误：RecursionError

记住：`property` 里应该返回真正存储数据的“底层属性”（常用写法是 `_name` 或 `__name`），不要再去访问同名的 property 本身。

## 继承：在原有类的基础上继续扩展

### 单继承：从一个父类“继承家产”

单继承就是一个子类只有一个父类，直接把父类的属性和方法“拿来用”。

In [8]:
class Person:
    home = "earth"      # 类属性

    def __init__(self, name):
        self.name = name

    def eat(self):
        print("eating...")

class YellowRace(Person):
    color = "yellow"    # 新增自己的类属性

y1 = YellowRace("张三")
print(y1.home)   # earth，来自父类
print(y1.color)  # yellow，来自子类
print(y1.name)   # 张三，父类构造函数里赋的值
y1.eat()         # eating...

earth
yellow
张三
eating...


单继承的好处是：公共能力（比如 `eat`、`home`）只在父类维护一份，所有子类自动拥有，写得少、改得也少。

### 多继承：一个类可以有多个父类

多继承就是一个类同时继承多个父类，调用方法时会按“从左到右”的顺序去父类里找。

In [9]:
class Person:
    home = "earth"

    def __init__(self, name):
        self.name = name

    def eat(self):
        print("eating...")

class YellowRace(Person):
    color = "yellow"

    def run(self):
        print("running...")

class Student(Person):
    def __init__(self, name, grade):
        self.name = name
        self.grade = grade

    def study(self):
        print("studying...")

class ChineseStudent(Student, YellowRace):
    country = "中国"

cs = ChineseStudent("张三", "三年级")
print(cs.home, cs.color, cs.country, cs.name, cs.grade)
cs.eat()
cs.run()
cs.study()

earth yellow 中国 张三 三年级
eating...
running...
studying...


多继承的好处是：可以把不同功能拆成不同类，然后“拼接”出一个既是学生、又是某个种族、还可能是某个角色的组合类。

### 在子类中调用父类方法：super()

子类重写了方法后，如果还想在里面顺带用一下父类的实现，可以用 `super()`。

In [10]:
class Person:
    def eat(self):
        print("eating...")

class Student(Person):
    def study(self):
        print("先吃再学")
        super().eat()      # 调用父类的 eat
        print("studying...")

s = Student()
s.study()

先吃再学
eating...
studying...


`super()` 的好处是：不用死写父类名，多继承时也能按正确顺序去找“上一层”的实现。

### 在子类中调用父类方法：直接写父类名

除了 `super()`，你也可以直接写 `父类名.方法(self)`，效果更直接。

In [11]:
class Person:
    def eat(self):
        print("eating...")

class Student(Person):
    def study(self):
        print("先吃再学")
        Person.eat(self)   # 手动把 self 传给父类方法
        print("studying...")

s = Student()
s.study()

先吃再学
eating...
studying...


这种写法好处是：一眼就能看出到底是从哪个父类里调用的哪个方法，在多继承层级比较复杂时更清晰。

### 方法解析顺序（MRO）：多继承里找方法的路线图

在多继承里，Python 会按照一条固定的顺序去父类中查找方法，这条顺序就叫方法解析顺序（MRO）。

In [12]:
class Person:
    def eat(self):
        print("Person.eat")

class YellowRace(Person):
    def eat(self):
        print("YellowRace.eat")

class Student(Person):
    def eat(self):
        print("Student.eat")

class ChineseStudent(Student, YellowRace):
    pass

print(ChineseStudent.__mro__)
cs = ChineseStudent()
cs.eat()

(<class '__main__.ChineseStudent'>, <class '__main__.Student'>, <class '__main__.YellowRace'>, <class '__main__.Person'>, <class 'object'>)
Student.eat


了解 MRO 的好处是：在多继承结构中，你能预判“到底会调用哪一个版本的方法”，避免踩坑。

### 方法重写：子类改写父类的行为

子类中定义了一个和父类同名的方法，调用时会优先用子类自己的，这就是“重写”。

In [13]:
class Person:
    def __init__(self, name):
        self.name = name

    def eat(self):
        print("用手吃东西")

class Chinese(Person):
    def __init__(self, name, area):
        super().__init__(name)   # 调用父类的 __init__
        self.area = area

    # 重写父类的 eat 方法
    def eat(self):
        print("用筷子吃")

c = Chinese("张三", "北京")
print(c.name, c.area)  # 张三 北京
c.eat()                # 用筷子吃

张三 北京
用筷子吃


重写的好处是：在保留父类通用逻辑的同时，子类可以按照自己的习惯自由定制行为。

## 多态：同一个接口，不同的实际表现

### 多态的基本用法

多态的意思是：同一个“调用方式”，传入不同对象，会表现出不同行为。

In [14]:
class Animal:
    def go(self):
        pass

class Dog(Animal):
    def go(self):
        print("跑")

class Fish(Animal):
    def go(self):
        print("游")

class Bird(Animal):
    def go(self):
        print("飞")

def go(animal):
    animal.go()   # 不关心具体是什么动物，只要有 go 方法就行

dog = Dog()
fish = Fish()
bird = Bird()

go(dog)   # 跑
go(fish)  # 游
go(bird)  # 飞

跑
游
飞


多态的好处是：调用方只需要“会用 go 这个接口”，不用关心具体是狗、鱼还是鸟，后续扩展新类型也不用改原来的调用代码。